# __Pinata Tool__

This notebook provides a quick overview for getting started with [Pinata Tool](/docs/integrations/tools/). For detailed documentation of all Pinata Tool features and configurations head to the [API reference](__api_ref_module__).

The Pinata Tool gives your agent the ability to manage files (upload, query and delete) on the IPFS Web3.0 Storage and manage groups (create, delete, get, list, update) for your files in your Pinata account.

## Overview

### Integration details

| Tool Class | Package | Python Support | Latest Package |
|:-----------|:--------|:---------------:|:---------------:|
| [PinataUploadFileTool](__api_ref_module__) | [@langchain/community](__api_ref_package__) | ❌ | ![NPM - Version](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |
| [PinataQueryFileTool](__api_ref_module__) | [@langchain/community](__api_ref_package__) | ❌ | ![NPM - Version](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |
| [PinataDeleteFileTool](__api_ref_module__) | [@langchain/community](__api_ref_package__) | ❌ | ![NPM - Version](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |
| [PinataCreateGroupTool](__api_ref_module__) | [@langchain/community](__api_ref_package__) | ❌ | ![NPM - Version](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |
| [PinataDeleteGroupTool](__api_ref_module__) | [@langchain/community](__api_ref_package__) | ❌ | ![NPM - Version](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |
| [PinataGetGroupTool](__api_ref_module__) | [@langchain/community](__api_ref_package__) | ❌ | ![NPM - Version](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |
| [PinataListGroupTool](__api_ref_module__) | [@langchain/community](__api_ref_package__) | ❌ | ![NPM - Version](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |
| [PinataUpdateGroupTool](__api_ref_module__) | [@langchain/community](__api_ref_package__) | ❌ | ![NPM - Version](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |

### Tool features

- **PinataUploadFileTool** – Uploads a file to Pinata and stores it on IPFS.
- **PinataQueryFileTool** – Retrieves metadata or information about a specific uploaded file.
- **PinataDeleteFileTool** – Deletes a previously uploaded file from Pinata.
- **PinataCreateGroupTool** – Creates a new group (or collection) to organize files in Pinata.
- **PinataDeleteGroupTool** – Deletes an existing group along with its associated metadata.
- **PinataGetGroupTool** – Retrieves detailed information about a specific group.
- **PinataListGroupTool** – Lists all existing groups under a Pinata account.
- **PinataUpdateGroupTool** – Updates metadata or configuration for an existing group.


## Setup

The integration lives in the `@langchain/community` package, along with the `pinata` dependency.

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/community @langchain/core pinata
</Npm2Yarn>
```

### Credentials

To use the tools, you will need to create an API key on [Pinata](https://pinata.cloud/) to obtain your JWT.

```typescript
process.env.PINATA_JWT="your-jwt-here"
```

It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability:

```typescript
process.env.LANGSMITH_TRACING="true"
process.env.LANGSMITH_API_KEY="your-api-key"
```

## Instantiation

You can import and instantiate an instance of the any Pinata tool like this:

In [ ]:
// An example of PinataUploadFileTool
import { PinataUploadFileTool } from "@langchain/community/tools/pinata"


const tool = new PinataUploadFileTool({
  pinataJwt: process.env.PINATA_JWT,
  pinataGateway: "your-custom-gateway-here",
});

## Invocation

### [Invoke directly with args](/docs/concepts/#invoke-with-just-the-arguments)


In [ ]:
await tool.invoke({
    url: "https://em-content.zobj.net/source/apple/391/astronaut-light-skin-tone_1f9d1-1f3fb-200d-1f680.png", //link to an astronaut emoji png
    name: "Astronaut Emoji",
    keyvalues: {"tag": "emoji"}
});

### [Invoke with ToolCall](/docs/concepts/#invoke-with-toolcall)

We can also invoke the tool with a model-generated `ToolCall`, in which case a `ToolMessage` will be returned:

In [ ]:
// This is usually generated by a model, but we'll create a tool call directly for demo purposes.
const modelGeneratedToolCall = {
  args: {
    url: "https://em-content.zobj.net/source/apple/391/astronaut-light-skin-tone_1f9d1-1f3fb-200d-1f680.png", //link to an astronaut emoji png
    name: "Astronaut Emoji",
    keyvalues: {"tag": "emoji"}
  },
  id: "1",
  name: tool.name,
  type: "tool_call",
}
await tool.invoke(modelGeneratedToolCall)

## Chaining

We can use our tool in a chain by first binding it to a [tool-calling model](/docs/how_to/tool_calling/) and then calling it:

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```


In [ ]:
// @lc-docs-hide-cell

import { ChatOpenAI } from "@langchain/openai"

const llm = new ChatOpenAI({
  model: "gpt-4o-mini",
})

In [ ]:
import { HumanMessage } from "@langchain/core/messages";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { RunnableLambda } from "@langchain/core/runnables";

const prompt = ChatPromptTemplate.fromMessages(
  [
    ["system", "You are a helpful assistant."],
    ["placeholder", "{messages}"],
  ]
)

const llmWithTools = llm.bindTools([tool]);

const chain = prompt.pipe(llmWithTools);

const toolChain = RunnableLambda.from(
  async (userInput: string, config) => {
    const humanMessage = new HumanMessage(userInput,);
    const aiMsg = await chain.invoke({
      messages: [new HumanMessage(userInput)],
    }, config);
    const toolMsgs = await tool.batch(aiMsg.tool_calls, config);
    return chain.invoke({
      messages: [humanMessage, aiMsg, ...toolMsgs],
    }, config);
  }
);

const toolChainResult = await toolChain.invoke("Can you help me to store a file as Astronaut Emoji on Pinata? Give it a tag as emoji. Here's the url: https://em-content.zobj.net/source/apple/391/astronaut-light-skin-tone_1f9d1-1f3fb-200d-1f680.png");

In [ ]:
const { tool_calls, content } = toolChainResult;

console.log("AIMessage", JSON.stringify({
  tool_calls,
  content,
}, null, 2));

## Agents

For guides on how to use LangChain tools in agents, see the [LangGraph.js](https://langchain-ai.github.io/langgraphjs/) docs.

## API reference

For detailed documentation of all Pinata Tool features and configurations head to the [API reference](__api_ref_module__)